# Bol.com Web Scraping

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

from tqdm import tqdm

In [2]:
categories = ['laptops', 'desktops', 'monitoren', 'randapparatuur', 'netwerk-&-internet', 'tassen-&-hoezen', 'printers-&-supplies', 'pc-gaming-winkel', 'software', 'dataopslag', 'beamers', 'componenten']

In [3]:
print(len(categories))

12


In [4]:
all_products = []

for i in (categories):
    for j in tqdm(range(1, 501)):
        response = requests.get(f'https://www.bol.com/nl/s/?page={j}&searchtext={i}&view=list').content
        soup = BeautifulSoup(response, 'lxml')
        products = soup.find_all(class_='product-item--row js_item_root')
        all_products.extend(products)
        time.sleep(2)

100%|██████████| 500/500 [21:23<00:00,  2.57s/it]


In [5]:
lst = []

for item in all_products:
    if item.find(class_='product-prices').getText() == '\nNiet leverbaar\n':
        price = None
    elif item.find(class_='star-rating')['title'] == 'Er zijn nog geen beoordelingen voor dit product.':
        score = None
    elif item.find(class_='star-rating')['title'] == 'Er zijn  nog geen beoordelingen voor dit product.':
        review = None
    else:
        product = item.find(class_='product-title--inline').a.getText()
        price = float(item.meta['content'])
        score = item.find(class_='star-rating')['title']
        review = item.find(class_='star-rating')['data-count']
        lst.append([product, price, score, review])

In [6]:
print(lst[:10])

[['ASUS Chromebook C523NA-A20209 - Chromebook - 15.6 inch', 259.0, 'klantbeoordeling: 4.4 van de 5 sterren', '67'], ['Lenovo Chromebook Ideapad 3-11IGL05 82BA000RMH - Chromebook - 11.6 Inch', 226.39, 'klantbeoordeling: 4.2 van de 5 sterren', '13'], ['Apple MacBook Air (April, 2020) MWTJ2N/A - 13.3 inch - Intel Core i3 - 256 GB - Spacegrijs', 885.0, 'klantbeoordeling: 4.8 van de 5 sterren', '14'], ['ASUS VivoBook 15 K512JP-BQ281T - Laptop - 15.6 Inch', 829.0, 'klantbeoordeling: 4.5 van de 5 sterren', '22'], ['Lenovo Ideapad L340-15IRH 81LK0150MH - Gaming Laptop - 15.6 Inch', 729.0, 'klantbeoordeling: 4.7 van de 5 sterren', '20'], ['ASUS Chromebook C523NA-EJ0055 - Chromebook - 15.6 Inch', 219.0, 'klantbeoordeling: 4.4 van de 5 sterren', '67'], ['Acer Swift 1 SF114-32-C7C0 - Laptop - 14 Inch', 379.0, 'klantbeoordeling: 4.0 van de 5 sterren', '13'], ['HP 17-by3740nd - Laptop - 17.3 Inch', 749.0, 'klantbeoordeling: 4.7 van de 5 sterren', '10'], ['Asus Chromebook C523NA-EJ0186 - Chromebook -

In [7]:
products_downloaded = len(lst)

In [8]:
df = pd.DataFrame(lst, columns=['Product', 'Price', 'Score', 'Reviews'])

In [9]:
len(df)

31306

In [10]:
df['Price'] = df['Price'].dropna()

In [11]:
len(df)

31306

In [12]:
df = df.drop_duplicates()

In [13]:
len(df)

6783

In [14]:
df = df.reset_index(drop=True)

In [15]:
df['Score'] = df['Score'].str.strip('klantbeoordeling: ')

In [16]:
df['Score'] = df['Score'].str[:3]

In [17]:
df['Score'] = df['Score'].astype(float)

In [18]:
df['Reviews'] = df['Reviews'].astype(int)

In [19]:
df.head(10)

,Product,Price,Score,Reviews
0,ASUS Chromebook C523NA-A20209 - Chromebook - 1...,259.00,4.4,67
1,Lenovo Chromebook Ideapad 3-11IGL05 82BA000RMH...,226.39,4.2,13
2,"Apple MacBook Air (April, 2020) MWTJ2N/A - 13....",885.00,4.8,14
3,ASUS VivoBook 15 K512JP-BQ281T - Laptop - 15.6...,829.00,4.5,22
4,Lenovo Ideapad L340-15IRH 81LK0150MH - Gaming ...,729.00,4.7,20
5,ASUS Chromebook C523NA-EJ0055 - Chromebook - 1...,219.00,4.4,67
6,Acer Swift 1 SF114-32-C7C0 - Laptop - 14 Inch,379.00,4.0,13
7,HP 17-by3740nd - Laptop - 17.3 Inch,749.00,4.7,10
8,Asus Chromebook C523NA-EJ0186 - Chromebook - 1...,299.00,4.4,67
9,Lenovo Ideapad 3 Chromebook 82C1000YMH - Chrom...,249.00,4.6,21


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6783 entries, 0 to 6782
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Product  6783 non-null   object 
 1   Price    6783 non-null   float64
 2   Score    6783 non-null   float64
 3   Reviews  6783 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 212.1+ KB


In [21]:
print('Products Downloaded:', products_downloaded, '\nDataset Size:', len(df))

Products Downloaded: 31306 
Dataset Size: 6783


In [22]:
df.to_csv('../data/processed/bol.csv')